# CIS Automation Tool for Windows Hardening
This notebook automates the **Audit**, **Remediation**, and **Validation** of security controls based on CIS benchmarks. It includes a **Dry-Run** mode for safe simulation without modifying the targets.

### 🛠️ Mode Selection
Set `DRY_RUN = True` to see what changes would be made without actually applying them.


In [2]:
import paramiko
import os
import platform
import json
import datetime
import time
import sys
from jinja2 import Template
from configs.cis_config import CIS_TASKS
from configs.control import ENABLED_STEPS

# GLOBAL CONFIGURATION
DRY_RUN = False  # Set to False to apply remediation for real.


## 2. Section: Utility Functions for Section Parsing
Implement the `parse_input_to_sections` function to convert user-defined strings (e.g., '1, 2, 5-10') into a set of integers for targeted hardening.


In [3]:
def parse_input_to_sections(selection_str):
    """
    แปลง string เช่น "1, 2, 17-19" หรือ "Section 1, Section 2" 
    ให้เป็น set ของตัวเลข [1, 2, 17, 18, 19]
    """
    if not selection_str or not selection_str.strip():
        return None  # ถ้า Input ว่าง ให้ถือว่าเลือกทั้งหมด
        
    target_sections = set()
    
    # ลบคำว่า "Section" และช่องว่างออก
    clean_str = selection_str.lower().replace("section", "").replace(" ", "")
    parts = clean_str.split(",")
    
    for part in parts:
        if not part: continue
        
        if "-" in part:
            # กรณีช่วง เช่น 17-19
            try:
                start, end = part.split("-")
                target_sections.update(range(int(start), int(end) + 1))
            except ValueError:
                print(f"Warning: รูปแบบผิดพลาด '{part}'")
        else:
            # กรณีตัวเลขเดี่ยว
            try:
                target_sections.add(int(part))
            except ValueError:
                print(f"Warning: ไม่ใช่ตัวเลข '{part}'")
                
    return target_sections


## 3. Section: CISAutomation Class with Dry-Run Support
The `CISAutomation` class is the core of this tool. It manages the SSH connection, executes remote PowerShell scripts, and records metrics.

**Modified logic for Dry-Run:**
- If `dry_run=True`, Step 2 (Remediation) is logged as "Skipping" without running the script.
- If `dry_run=True`, Step 3 (Final Audit) is skipped.
- Both Step 1 (Initial Audit) and Step 3 (if applicable) can be independently enabled or disabled via `configs/control.py`.


In [4]:
class CISAutomation:
    def __init__(self, host_config, target_sections=None, dry_run=False):
        self.host = host_config["host"]
        self.user = host_config["user"]
        self.password = host_config["password"]
        self.target_sections = target_sections 
        self.dry_run = dry_run # Dry-run flag
        
        # Setup directories
        self.log_dir, self.report_dir = "logs", "reports"
        for d in [self.log_dir, self.report_dir]:
            if not os.path.exists(d): os.makedirs(d)

        self.host_report_dir = os.path.join(self.report_dir, self.host.replace(" ", "_"))
        self.host_log_dir = os.path.join(self.log_dir, self.host.replace(" ", "_"))
        for d in [self.host_report_dir, self.host_log_dir]:
            if not os.path.exists(d): os.makedirs(d)
        
        current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        self.log_file = os.path.join(self.host_log_dir, f"{current_time}.log")
        
        self.ssh = paramiko.SSHClient()
        self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.sftp = None
        
        self.results = {
            "host_ip": self.host, "timestamp": current_time, "tasks": [],
            "summary": { "step1_pass": 0, "step1_fail": 0, "step2_pass": 0, "step2_fail": 0, "step3_pass": 0, "step3_fail": 0, "total_tasks": 0, "completed_tasks": 0 }
        }
    
    def log(self, message, is_script_output=False):
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        formatted_msg = "\n".join([f"      | {line}" for line in message.split('\n') if line.strip()]) if is_script_output else f"[{timestamp}] {message}"
        print(formatted_msg)
        with open(self.log_file, "a", encoding="utf-8") as f: f.write(formatted_msg + "\n")
    
    def connect(self):
        try:
            self.ssh.connect(hostname=self.host, username=self.user, password=self.password, timeout=10)
            self.sftp = self.ssh.open_sftp()
            
            # ++++++++++ เพิ่มโค้ดส่วนนี้เพื่อโยน LGPO.exe ไปรอไว้ที่เครื่องปลายทาง ++++++++++
            try:
                self.sftp.put("tools/LGPO.exe", "C:/Windows/Temp/LGPO.exe")
            except Exception as e:
                self.log(f"Warning: ไม่สามารถอัปโหลด LGPO.exe ได้: {str(e)}")
            # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
            
            return True
        except Exception as e:
            self.log(f"ERROR: ไม่สามารถเชื่อมต่อกับ {self.host}: {str(e)}")
            return False

    def close_connections(self):
        try:
            if self.sftp: self.sftp.close()
            if self.ssh: self.ssh.close()
        except Exception: pass
    
    def exec_remote_script(self, local_path):
        """รันสคริปต์บนเครื่องปลายทาง"""
        remote_temp = "C:/Windows/Temp/cis_temp_exec.ps1"
        if not os.path.exists(local_path): return 404, f"Local file not found: {local_path}"
        if self.sftp is None: return 500, "SFTP session is not established."
        
        try:
            self.sftp.put(local_path, remote_temp)
            _, stdout, stderr = self.ssh.exec_command(f"powershell.exe -ExecutionPolicy Bypass -File {remote_temp}")
            exit_status = stdout.channel.recv_exit_status()
            combined_output = stdout.read().decode().strip() + "\n" + stderr.read().decode().strip()
            return exit_status, combined_output.strip()
        except Exception as e: return 500, f"Execution error: {str(e)}"
    
    def _execute_step(self, task_id, step_key, script_type, task_result):
        step_num = step_key[-1]
        step_status_key = f"step{step_num}_status"
        step_output_key = f"step{step_num}_output"
        summary_pass_key = f"step{step_num}_pass"
        summary_fail_key = f"step{step_num}_fail"
        
        script_file = f"./audit/a{task_id}.ps1" if script_type == "audit" else f"./remedtation/r{task_id}.ps1"
        
        # DRY RUN LOGIC
        if self.dry_run:
            if script_type == "remedtation":
                self.log(f"  [DRY RUN] Would execute remediation: {script_file}")
                task_result[step_status_key] = "SKIP (DRY)"
                task_result[step_output_key] = "DRY RUN: No modification performed."
                return
            if step_num == "3":
                self.log("  [DRY RUN] Skipping step 3 (Final audit)")
                return

        exit_code, output = self.exec_remote_script(script_file)
        if output: self.log(output, is_script_output=True)
        
        step_status = "PASS" if exit_code == 0 else "FAIL"
        task_result[step_status_key] = step_status
        task_result[step_output_key] = output
        
        if step_status == "PASS": self.results["summary"][summary_pass_key] += 1
        else: self.results["summary"][summary_fail_key] += 1
        self.log(f"    -> {step_status}")
    
    def _determine_final_status(self, task_result):
        if ENABLED_STEPS.get("step3"): return task_result["step3_status"]
        if ENABLED_STEPS.get("step2"): return task_result["step2_status"]
        return task_result["step1_status"]
    
    def _get_status_icon(self, status):
        return {"PASS": "[PASS]", "FAIL": "[FAIL]", "SKIP (DRY)": "[DRY SKIP]"}.get(status, "[SKIP]")
    
    def _process_task(self, task, task_idx, total_tasks):
        task_id, task_desc = task["id"], task["desc"]
        self.log(f"\n[{task_idx}/{total_tasks}] CONTROL: {task_id} - {task_desc}")
        
        task_result = { "id": task_id, "description": task_desc, "step1_status": "SKIP", "step1_output": "", "step2_status": "SKIP", "step2_output": "", "step3_status": "SKIP", "step3_output": "", "row_class": "even" if task_idx % 2 == 0 else "odd" }

        # STEP 1
        if ENABLED_STEPS.get("step1", False): self._execute_step(task_id, "step1", "audit", task_result)
        # STEP 2
        if ENABLED_STEPS.get("step2", False): self._execute_step(task_id, "step2", "remedtation", task_result)
        # STEP 3
        if ENABLED_STEPS.get("step3", False): self._execute_step(task_id, "step3", "audit", task_result)
        
        self.results["tasks"].append(task_result)
        self.results["summary"]["completed_tasks"] += 1
        
        final_status = self._determine_final_status(task_result)
        self.log(f"*** {task_id} Finished: {self._get_status_icon(final_status)}")

    def run_flow(self):
        enabled_tasks = []
        for t in CIS_TASKS:
            if not t["enabled"]: continue
            if self.target_sections is None: enabled_tasks.append(t); continue
            try:
                task_section = int(t['id'].split('.')[0])
                if task_section in self.target_sections: enabled_tasks.append(t)
            except (ValueError, IndexError): continue
        total_tasks = len(enabled_tasks)
        self.results["summary"]["total_tasks"] = total_tasks
        if total_tasks == 0: return self.results

        if not self.connect():
            self.log(f"ERROR: ไม่สามารถเชื่อมต่อกับ {self.host}")
            return self.results
        
        for task_idx, task in enumerate(enabled_tasks, 1):
            self._process_task(task, task_idx, total_tasks)
            self._process_task(task, task_idx, total_tasks)
        
        self.close_connections()
        return self.results


## 4. Section: SSH Connection Validation (Pre-flight Check)
Ensures SSH connectivity for all target hosts before starting the workflow.


In [5]:
def validate_all_connections(hosts):
    """
    ตรวจสอบการเชื่อมต่อ SSH ของทุกโฮสต์ก่อนเริ่มทำงาน
    Returns: True ถ้าผ่านหมด, False ถ้ามีเครื่องใดเครื่องหนึ่งไม่ผ่าน
    """
    print(f"\n{'='*80}")
    print(f"PRE-FLIGHT CHECK: Checking SSH Connectivity for {len(hosts)} hosts...")
    print(f"{'='*80}")
    
    all_connected = True
    failed_hosts = []

    for i, host_config in enumerate(hosts, 1):
        print(f"[{i}/{len(hosts)}] Checking {host_config['host']}...", end=" ")
        
        # สร้าง Instance ชั่วคราวเพื่อเทส connection
        temp_automator = CISAutomation(host_config)
        
        if temp_automator.connect():
            print("OK ✓")
            temp_automator.close_connections()
        else:
            print("FAILED ✗")
            all_connected = False
            failed_hosts.append(host_config['host'])
    
    print("-" * 80)
    
    if not all_connected:
        print("\n[!!!] CRITICAL ERROR: Connection check failed for the following hosts:")
        for fh in failed_hosts:
            print(f" - {fh}")
        print("\nStopping execution because 'SSH Check Pass All' condition was not met.")
        return False
    
    print("\n[✓] All hosts are reachable. Proceeding to automation tasks.")
    return True


## 6. Section: Interactive Report Generation
Utilize Jinja2 templates to generate summary reports based on individual host results.


In [6]:
def generate_summary_report(all_results):
    template_path = "templates/summary_templatev1.html"
    if not os.path.exists(template_path):
        print(f"ERROR: Template not found: {template_path}")
        return None
    try:
        with open(template_path, "r", encoding="utf-8") as f: template_content = f.read()
        template = Template(template_content)
        total_summary = { "total_hosts": len(all_results), "total_tasks": 0, "step1_pass": 0, "step1_fail": 0, "step2_pass": 0, "step2_fail": 0, "step3_pass": 0, "step3_fail": 0 }
        
        for result in all_results:
            for k in ["total_tasks", "step1_pass", "step1_fail", "step2_pass", "step2_fail", "step3_pass", "step3_fail"]:
                total_summary[k] += result["summary"][k]
        
        html_content = template.render(hosts=all_results, summary=total_summary, timestamp=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        summary_dir = "reports/summary"
        os.makedirs(summary_dir, exist_ok=True)
        summary_file = os.path.join(summary_dir, f"summary_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.html")
        with open(summary_file, "w", encoding="utf-8") as f: f.write(html_content)
        print(f"\nSummary Report Generated: {summary_file}")
        return summary_file
    except Exception as e: print(f"ERROR: {str(e)}"); return None


## 5. Section: Execution Orchestration for Multiple Hosts
Here we define the main execution logic that prompts for sections and iterates through each host. It also respects the `DRY_RUN` flag.


In [8]:
def run_all_hosts_interactive():
    config_file = "configs/hosts.json"
    if not os.path.exists(config_file):
        print(f"ERROR: Config file not found: {config_file}")
        return
    with open(config_file, "r") as f:
        hosts = json.load(f).get("hosts", [])
    
    if not hosts:
        print("No hosts configured")
        return
        
    print(f"\n{'='*80}")
    print("CIS SELECTION MENU")
    print(f"{'='*80}")
    print("ตัวอย่าง: 1, 2, 5 (เฉพาะ section นั้น) หรือ 17-19 (ช่วง) หรือกด Enter เพื่อรันทั้งหมด")
    section_input = input("เลือก Section ที่ต้องการรัน: ")
    target_sections = parse_input_to_sections(section_input)
    
    # 1. Pre-flight Check
    if not validate_all_connections(hosts):
        return

    # 2. Start Main Hub
    all_results = []
    print(f"\nDRY RUN MODE: {DRY_RUN}\n")
    
    try:
        for i, host_config in enumerate(hosts, 1):
            print(f"\n[เครื่องที่ {i}/{len(hosts)}] {host_config['host']}")
            automator = CISAutomation(host_config, target_sections=target_sections, dry_run=DRY_RUN)
            if automator.connect():
                results = automator.run_flow()
                all_results.append(results)
            else:
                print(f"✗ Failed for {host_config['host']}")

    except KeyboardInterrupt:
        print("\n[!] Execution stopped by user.")
        return

    if all_results:
        generate_summary_report(all_results)

# RUN THE TOOL!
run_all_hosts_interactive()



CIS SELECTION MENU
ตัวอย่าง: 1, 2, 5 (เฉพาะ section นั้น) หรือ 17-19 (ช่วง) หรือกด Enter เพื่อรันทั้งหมด

PRE-FLIGHT CHECK: Checking SSH Connectivity for 1 hosts...
[1/1] Checking 192.168.150.153... OK ✓
--------------------------------------------------------------------------------

[✓] All hosts are reachable. Proceeding to automation tasks.

DRY RUN MODE: False


[เครื่องที่ 1/1] 192.168.150.153
[16:32:05] 
[1/80] CONTROL: 18.10.9.1.1 - Ensure 'Configure enhanced anti-spoofing' is set to 'Enabled' (Automated)
      | ==============================================================
      | Audit started: 2026-02-19 16:32:07
      | Control 18.10.9.1.1: Ensure 'Configure enhanced anti-spoofing' is Enabled
      | ==============================================================
      | Value is incorrect (0). Expected: 1 (Enabled).
      | ==============================================================
      | Audit completed at 02/19/2026 16:32:08
      | Status: NON-COMPLIANT
      | ==